# CWT hyperparameters tuning

Use GPU T4 x 2  
When using GP100, there are XLA errors.  

5 channels (LT, RT, LP, RP, C).

Implementing tf.keras.metrics.KLDivergence().



In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers, regularizers
import keras_tuner as kt
import sys
import os

# Sets off SettingWithCopyWarning.
pd.set_option('mode.chained_assignment', None)


# ----------------------------------------
# Flags for working on my different machines.
flag_kaggle = True
# flag_FW = True
# flag_LN = True

try:
    if flag_kaggle:
        sys.path.insert(0, '/kaggle/input/hms-lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
        output_dir = ''
except:
    pass 

try:
    if flag_FW:
        sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 

try:
    if flag_LN:
        sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 
# ----------------------------------------

from KLmetric import score

path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

2024-03-14 05:16:47.328830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 05:16:47.328929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 05:16:47.625775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data generators

In [2]:
#
# Data generator for training.
#
# coefficients of cwt's arrays
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y


#
# Test Data generator for predicting
# 

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X


In [3]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 12192
Observations in validation set: 2176


## HP tuning

In [4]:
input_shape = training_generator.get_dim()
num_classes = 6

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))

    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Conv2D(
        filters=hp.Int(f"units_{i}", min_value=32, max_value=64, step=32),
        kernel_size = hp.Choice(f"kernel_{i}", [3,5]),
        activation='relu', padding="same"))
        if hp.Boolean(f"dropout_{i}"):
            model.add(layers.Dropout(rate=0.25))
        if hp.Boolean(f"pooling_{i}"):
            model.add(layers.MaxPooling2D((3, 3)))

    if hp.Boolean(f"normalization"):
        model.add(layers.BatchNormalization())
    
    model.add(layers.Flatten())
    model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )

    model.add(layers.Dense(num_classes, activation='softmax'))

    # learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', values=[0.2, 0.1, 0.01, 0.005])

    opt = keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=momentum,
        )
    
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.KLDivergence(),
        metrics=[tf.keras.metrics.KLDivergence()])

    return model



In [5]:
tuner = kt.Hyperband(model_builder,
                     # objective=kt.Objective("val_kullback_leibler_divergence", direction="min"),
                     objective=kt.Objective("val_kl_divergence", direction="min"),  # In Kaggle.
                     max_epochs=30,
                     factor=3,
                     directory=output_dir,
                     project_name='cwt_v1',
                     overwrite=True)


In [6]:
# tuner = kt.RandomSearch(
#     hypermodel=model_builder,
#     # objective="val_accuracy",
#     objective=kt.Objective("val_mean_absolute_error", direction="min"),
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="my_dir",
#     project_name="helloworld",
# )


In [7]:
# tuner.search_space_summary()

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_kl_divergence', patience=5)


In [9]:
tuner.search(training_generator, epochs=3, validation_data=validation_generator, callbacks=[stop_early])

Trial 90 Complete [00h 08m 41s]
val_kl_divergence: 0.9247735738754272

Best val_kl_divergence So Far: 0.5162684321403503
Total elapsed time: 02h 54m 25s


In [10]:
models = tuner.get_best_models()
best_model = models[0]
best_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 49, 400, 64)    │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 133, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 133, 64)    │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 133, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 136192)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     8,716,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,822,406 (33.65 MB)

 Trainable params: 8,822,278 (33.65 MB)

 Non-trainable params: 128 (512.00 B)

In [11]:
tuner.results_summary(num_trials=1)

Results summary
Results in ./cwt_v1
Showing 1 best trials
Objective(name="val_kl_divergence", direction="min")

Trial 0050 summary
Hyperparameters:
num_layers: 2
units_0: 64
kernel_0: 3
dropout_0: False
pooling_0: True
normalization: True
activation: tanh
learning_rate: 0.01
momentum: 0.005
units_1: 64
kernel_1: 5
dropout_1: False
pooling_1: False
units_2: 64
kernel_2: 5
dropout_2: False
pooling_2: False
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0046
Score: 0.5162684321403503


## Retrain the model

In [12]:
checkpoint_filepath = f'{output_dir}checkpoint-12-1.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_kl_divergence',  # Name in Kaggle.
#     monitor='val_kullback_leibler_divergence',  # Name in FW.
    mode='min',
    save_best_only=True)

best_hps=tuner.get_best_hyperparameters()[0]

model = tuner.hypermodel.build(best_hps)

history = model.fit(training_generator, epochs=5,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])


Epoch 1/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 15s 35ms/step - kl_divergence: 1.4914 - loss: 1.4914 - val_kl_divergence: 1.1206 - val_loss: 1.1202
Epoch 2/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - kl_divergence: 1.0690 - loss: 1.0690 - val_kl_divergence: 0.9316 - val_loss: 0.9375
Epoch 3/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - kl_divergence: 0.8802 - loss: 0.8802 - val_kl_divergence: 0.8261 - val_loss: 0.8321
Epoch 4/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - kl_divergence: 0.6723 - loss: 0.6723 - val_kl_divergence: 0.6916 - val_loss: 0.6916
Epoch 5/5
381/381 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - kl_divergence: 0.4611 - loss: 0.4610 - val_kl_divergence: 0.6016 - val_loss: 0.6014


## Scoring without submission

Using a local test set.

In [13]:
loaded_model = keras.models.load_model(checkpoint_filepath)


In [14]:
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)

y_pred = loaded_model.predict(test_generator)

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [15]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

test_items = np.load(path_test_items)
df_test_items = pd.DataFrame(test_items)
df_test_items[0] = df_test_items[0].astype(int)

sub = pd.DataFrame({'eeg_id':df_test_items[0]})
sub[TARGETS] = np.round(y_pred,6)
sub.to_csv('submission.csv',index=False)

df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
df_test_scoring.columns = sub.columns

score(df_test_scoring, sub, 'eeg_id')

0.5771306181490531